In [1]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import cv2
from tqdm import tqdm

from scipy import spatial
from skimage.measure import compare_ssim as SSIM
from keras.layers import Flatten, Dense, Input,concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout
from keras.models import Model
from keras.models import Sequential
import tensorflow as tf

In [2]:
vgg16 = keras.applications.VGG16(weights = 'imagenet', include_top = True, pooling = 'max', input_shape = (224, 224, 3))
basemodel = Model(inputs = vgg16.input, outputs = vgg16.get_layer('fc2').output)

def vector_extractor(img, SZ):
    feature_vector = basemodel.predict(img.reshape(1, SZ[0], SZ[1], 3))
    return feature_vector

def similarity_calculator(vector1, vector2):
    return 1 - spatial.distance.cosine(vector1, vector2)

In [7]:
filepath = "D://A NDSC 2020//test set//product-matching-id-ndsc-2020//" #<----- ***RENAME THIS PATH***
test = pd.read_csv(filepath + "new_test_set.csv") #<----- ***RENAME THIS PATH***
test

,pair_index,title_1,image_1,title_2,image_2
0,0,Celana Chino Panjang Original Motz,6985a78ebd6b7d9f120f852b412bba5b.jpg,Celana Chino Panjang Original Motz,81f8fab965d009483d004af0c697dd97.jpg
1,1,[BISA COD] Tas LOL Boneka Flash Anak Ransel,4d6f929847cbe632b15baa556646a414.jpg,Bisa Cod Tas Lol Boneka Mewah Anak Ransel Tas ...,1e2facde94db9de3037741ec4a5313c8.jpg
2,2,KOYO AJAIB KINOKI GOLD paket Standard,e21056441f03442737a2c074a4ab232d.jpg,BL - V.PJG Baju Kaos Vneck Polos Lengan Panjan...,7b91ab6c71c6e9ed432201f12379509c.jpg
3,3,Zwitsal Natural Aloe Vera Baby Bath 2 In 1 Hai...,3ac880adf8ffbcf23846554efc052694.jpg,ZWITSAL Natural Baby Bath 2 in 1 Hair & Body b...,46b00a7c6a869e00cad8cc08fb56a2f7.jpg
4,4,TAS IMPORT JAKARTA CLASSIC DIAMOND JELLY MATTE...,4ddd79c9d601bd25c1a56e0a10fb5708.jpg,TAS IMPORT JAKARTA CLASSIC DIAMOND JELLY MATTE...,10a97ae389e298f72bc455aa4cb4fba7.jpg
...,...,...,...,...,...
32575,32575,INBEX Soundplus In-ear Earphone with Mic,786ac779816e9bc94eb91c76452d3510.jpg,In-ear Wired Earphone 5 Colors 3.5mm In Ear Sp...,a7d840c66c1669dae9ef82c25a7caa46.jpg
32576,32576,Kerudung Segiempat Polos BELLA SQUARE Hijab Po...,12c3fb43189a1ed9850e970ec4d1821d.jpg,Casing Soft Case TPU untuk oppo A9 2020 A5 202...,7c0b148d2433bada3c50e05d2349085a.jpg
32577,32577,Good Day Moccacino 20gr PROMO !!!,c59eb714f08592358d415059ce0c974d.jpg,Mujigae Spicy Rapokki (Ramen Topokki) 350 gr,743e031062cc8d89944e63867a5b2b13.jpg
32578,32578,Kent Long Jeans Black Old,77af94418b631a963b50cdf7fd0ac037.jpg,Kent Long Jeans Black Old / Celana Jeans Pria ...,856c5e7e874402cdacad503950892b10.jpg


In [9]:
path = "D://A NDSC 2020//test set//product-matching-id-ndsc-2020//test_img//test_img//"  #<----- ***RENAME THIS PATH***
SZ = (224, 224)
VGG_sim = []
for n in tqdm(range(24000, len(test.values))):
    photo1 = cv2.resize(cv2.imread(path + test.image_1[n]), SZ)
    photo2 = cv2.resize(cv2.imread(path + test.image_2[n]), SZ)
    VGG_sim.append(similarity_calculator(vector_extractor(photo1, SZ), vector_extractor(photo2, SZ)))

100%|██████████| 8580/8580 [1:24:30<00:00,  1.69it/s]


In [10]:
pd.DataFrame({'VGG':VGG_sim}).to_csv('VGG_weight.csv', index = False) #<----- ***RENAME THIS FILENAME***